### import ライブラリ

In [1]:
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap


# アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


* ブレンディング
* バギング
* スタッキング

## 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中の`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [2]:
df= pd.read_csv('../house-prices-advanced-regression-techniques/train.csv')
X = df[['GrLivArea', 'YearBuilt']]
target = df.SalePrice

In [3]:
X = X.values
target = target.values

In [4]:
std = StandardScaler()
target = np.log(target)

In [5]:
target = np.ravel(target)

In [6]:
# データ分割
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)


In [7]:
X.shape, X_train.shape, X_test.shape

((1460, 2), (1168, 2), (292, 2))


## scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


[sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

# ブレンディング

## 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [8]:
# 同じ手法でパラメータを変えたインスタンスの組み合わせ
dtr_1 = DecisionTreeRegressor(splitter='random')
dtr_1.fit(X_train, y_train)
dtr_pred_1 = dtr_1.predict(X_test)

dtr_2 = DecisionTreeRegressor(splitter='best')
dtr_2.fit(X_train, y_train)
dtr_pred_2 = dtr_2.predict(X_test)

# ブレンド
blend = np.stack([dtr_pred_1, dtr_pred_2], 1)
blend_pred = np.average(blend, axis=1)

# 比較
'dtr_1', mean_squared_error(y_test, dtr_pred_1), 'dtr_2', mean_squared_error(y_test, dtr_pred_2), 'dtr_1, dtr_2', mean_squared_error(y_test, blend_pred)

('dtr_1',
 0.06511442696822994,
 'dtr_2',
 0.06586510328914023,
 'dtr_1, dtr_2',
 0.05465309467360677)

In [39]:
# ２つの異なる手法のブレンド

leg = LinearRegression()
leg.fit(X_train, y_train)
leg_pred = leg.predict(X_test)

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr_pred = dtr.predict(X_test)

# ブレンド
blend = np.stack([leg_pred, dtr_pred], 1)
blend_pred = np.average(blend, axis=1)

# 比較
'leg', mean_squared_error(y_test, leg_pred), 'dtr', mean_squared_error(y_test, dtr_pred), 'leg, dtr', mean_squared_error(y_test, blend_pred)

('leg',
 0.05846683591964387,
 'dtr',
 0.07841405750997235,
 'leg, dtr',
 0.056400550221579936)

In [77]:
# ブレンドの比率を調整する
leg = LinearRegression()
leg.fit(X_train, y_train)
leg_pred = leg.predict(X_test)

svr = SVR(gamma='scale')
svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr_pred = dtr.predict(X_test)

# ブレンド
blend = np.stack([leg_pred, svr_pred, dtr_pred], 1)
blend_pred = np.average(blend, axis=1)

# ブレンド2
rate = [0.1, 0.8, 0.1]
blend_2 = np.stack([leg_pred*rate[0], svr_pred*rate[1], dtr_pred*rate[2]], 1)
blend_pred_2 = np.sum(blend_2, axis=1)

# 比較
'blend_1', mean_squared_error(y_test, blend_pred), 'blend_2', mean_squared_error(y_test, blend_pred_2)

('blend_1', 0.047575620801935045, 'blend_2', 0.041800499855544605)

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

## バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ **ブートストラップサンプル** ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation]()


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [11]:
round(X.shape[0]*0.8)

1168

In [78]:
# データ分割
n = 3
row = int(X.shape[0]*0.2)
pred_list = np.zeros((row, n))

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, shuffle=True)

for i in range(n):
    index = np.random.choice(len(X_train), int(len(X_train)*0.8))
    dtr = DecisionTreeRegressor(random_state=42)
    dtr.fit(X_train[index], y_train[index])
    pred_list[:,i] = dtr.predict(X_test)
    
avg = np.average(pred_list, axis=1)

pred_list = np.hstack((pred_list, avg.reshape(-1, 1)))


print(0, mean_squared_error(y_test, pred_list[:,0]))
print(1, mean_squared_error(y_test, pred_list[:,1]))
print(2, mean_squared_error(y_test, pred_list[:,2]))
print('avg', mean_squared_error(y_test, pred_list[:,3]))

0 0.064415099440627
1 0.08360751834300002
2 0.06555633653447579
avg 0.048255137396867015


# スタッキング

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

## スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
$K_{0}=3,M_{0}=2$程度にします。

### 《学習時》


（ステージ 0）


* $学習データを K_{0}個に分割する。$
* $分割した内の (K_{0}−1)個をまとめて学習用データ、残り 1 個を推定用データとする組み合わせが K_{0}個作れる。$
* $あるモデルのインスタンスを K_{0} 個用意し、異なる学習用データを使い学習する。$
* $それぞれの学習済みモデルに対して、使っていない残り 1 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）$
* $さらに、異なるモデルのインスタンスも K_{0} 個用意し、同様のことを行う。モデルが M_{0}個あれば、 M_{0} 個のブレンドデータが得られる。$


（ステージ n ）


* $ステージ n−1のブレンドデータをM_{n}−1 次元の特徴量を持つ学習用データと考え、 K_{¥n}個に分割する。以下同様である。$

（ステージ N ） ＊最後のステージ


* $ステージ N−1の M_{N−1} 個のブレンドデータをM_{N−1}次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。$

### 《推定時》


（ステージ 0）


* テストデータを K_{0}× M_{0} 個の学習済みモデルに入力し、K_{0} × M_{0} 個の推定値を得る。これを K_{0} の軸で平均値を求め M_{0}次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）$

（ステージ n）


* $ステージ n−1で得たブレンドテストを K_{n}×M_{n}個の学習済みモデルに入力し、K_{n}×M_{n}個の推定値を得る。
これを K_{n}の軸で平均値を求め M_{0}次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）$

（ステージ N ）＊最後のステージ

* $ステージ N−1で得たブレンドテストを学習済みモデルに入力し、推定値を得る。$

In [76]:
# stage 0
y = target
n = 10
models = [DecisionTreeRegressor(), LinearRegression(), SVR(gamma='scale')]
kf = KFold(n_splits=n, shuffle=True, random_state=42)

cnt = 0
cnt_model = 0
blend_data = np.zeros((len(X_train), len(models)))
# val_data = np.zeros((len(X_train), len(models)))
blend_test = np.zeros((len(models), len(X_test), n))
for i in range(len(models)):
    predict_list = np.array([])
    val_list =  np.array([])
    for train_index, test_index in kf.split(X_train):
        X_bd_train, X_bd_test = X_train[train_index], X_train[test_index]
        y_bd_train, y_bd_test = y_train[train_index], y_train[test_index]
        models[i] 
        models[i].fit(X_bd_train, y_bd_train)
        bd_pred = models[i].predict(X_bd_test)
        bt_pred = models[i].predict(X_test)
        predict_list = np.append(predict_list, bd_pred)
#         val_list = np.append(val_list, y_bd_test)
        blend_test[cnt_model, :, cnt] = bt_pred
        cnt += 1                    
    blend_data[:,i] = predict_list
#     val_data = val_list
    cnt_model += 1
    cnt = 0



# stage n
blend_data_2 = np.array([])
for train_index, test_index in kf.split(X_train):
    X_bd_train_2, X_bd_test_2 = blend_data[train_index], blend_data[test_index]
    y_bd_train_2, y_bd_test_2 = val_data[train_index], val_data[test_index]
    model = models[0] 
    model.fit(X_bd_train_2, y_bd_train_2)
    bd_pred_2 = model.predict(X_bd_test_2)
    blend_data_2 = np.append(blend_data_2, bd_pred_2)

blend_test = np.average(blend_test, axis=2).T # ブレンドテスト



# last stage
last_model = models[2] 
last_model.fit(X_bd_train_2, y_bd_train_2)
last_pred = last_model.predict(blend_test)

'second_model', mean_squared_error(y_test, last_pred)

('second_model', 0.041554366516274324)